Para importar as bibliotecas

In [19]:
import pandas as pd
import qrcode
import os

from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.oauth2 import service_account

Para gerar as pastas e os páginas HTML

In [20]:

# Configurações da API do Google Drive
SERVICE_ACCOUNT_FILE = 'caminho/para/credenciais.json'
SCOPES = ['https://www.googleapis.com/auth/drive.file']

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('drive', 'v3', credentials=credentials)

def upload_file_to_drive(file_path, mime_type='image/jpeg'):
    file_metadata = {'name': file_path.split('/')[-1]}
    media = MediaFileUpload(file_path, mimetype=mime_type)
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    file_id = file.get('id')
    return f'https://drive.google.com/uc?id={file_id}'

base_html = pd.read_excel('Cuidado_com_os_dados.xlsx',
                              usecols=['Nome','Turmas', 'Sala', 'Professor', 'Inscrição', 'Jantar', 'Notebook', 'Imagem'])

# for i in range(len(base_html)):
for i in range(len(base_html)):
    
    image_link = upload_file_to_drive(base_html['imagem'])
    
    site = f"""
    <!DOCTYPE html>
    <html lang="pt-br">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Germinare TECH</title>
        <style>
            @import url('https://fonts.googleapis.com/css2?family=Inter:wght@100..900&display=swap');

            * {{
                margin: 0;
                padding: 0;
                box-sizing: border-box;
                font-family: 'Inter', sans-serif;
                font-size: 25px;

            }}

            body {{
                background-color: #fdfdfd;
                color: #333;
                height: 100vh;
                display: flex;
                align-items: center;
                justify-content: center;
            }}

            main {{
                padding: .5rem;
                width: fit-content;
                background-color: #CCD0E7;
                border-radius: .6rem;
                align-items: center;
                display: flex;
                flex-direction: column;
            }}

            img {{
                height: 10rem;
                border-radius: .6rem;
            }}

            h1 {{
                text-align: center;
                margin-top: .5rem;
            }}

            p {{
                margin-top: .5rem;
                font-size: .8rem;
            }}

            @media only screen and (max-width: 600px) {{
                img {{
                    height: 6rem;
                }}

                h1 {{
                    font-size: .7rem;
                }}

                p {{
                    font-size: .6rem;
                }}
            }}

            @media only screen and (max-width: 350px) {{
                main {{
                    margin-inline: 1rem;
                }}
                img {{
                    height: 4rem;
                }}

                h1 {{
                    font-size: .6rem;
                }}

                p {{
                    font-size: .5rem;
                }}

            }}
        </style>
    </head>

    <body>
        <main>
            <img src="{image_link}" alt="Imagem do candidato">
            <section id="info">
                <h1>Informações do Candidato</h1>
                <p>
                    Nome: {base_html["Nome"][i]}
                </p>
                <p>
                    Turma: {base_html["Turmas"][i]}
                </p>
                <p>
                    Sala: {base_html["Sala"][i]}
                </p>
                <p>
                    Professor: {base_html["Professor"][i]}
                </p>
                <p id="notebook">
                    Notebook: {base_html["Notebook"][i]}
                </p>
            </section>
        </main>

        <script>
            const notebook = document.getElementById('notebook')
            if (notebook.innerText.toLowerCase().includes('sim')) {{
                {{
                    notebook.style.color = 'green'
                }}
            }}
            else {{
                {{
                    notebook.style.color = 'red'
                }}
            }}
        </script>
    </body>
    </html>
        """
    # print(info)


    folder = os.path.join("paginas_html/", base_html["Inscrição"][i])
    os.mkdir(folder)
    # arquivo = open(f'{folder}/{base_html["Inscrição"][i]}.html', "a")
    with open(f'{folder}/{base_html["Inscrição"][i]}.html', "w", encoding="utf-8") as arquivo:
    #arquivo = open(f'teste.html', "a")

        # print(site)
        arquivo.write(site)
        arquivo.close()

Para gerar os QR_Codes

In [22]:
df_qr = pd.read_excel('Cuidado_com_os_dados.xlsx',
                              usecols=['Nome','Turmas', 'Sala', 'Professor', 'Inscrição', 'Imagem'])

# for i in range(len(df_qr)):
for i in range(len(df_qr)):
    nome_insc = ""
    link = f'https://gedaijef.github.io/qrcode/paginas_html/{df_qr["Inscrição"][i].replace(" ", "_")}/{df_qr["Inscrição"][i].replace(" ", "_")}.html'
    qrimg = qrcode.make(link)
    qrimg.save(f'qrCodes/{df_qr["Inscrição"][i].replace(" ", "_")}.png')